In [ ]:
import pandas as pd
import numpy as np

In [ ]:
tes = pd.read_csv("../../data/input/train_with_meta.csv")

In [ ]:
tes.iloc[:, 8:]

In [ ]:
pseudo_0 = pd.read_csv("../../data/pseudo/pseudo_fold_0.csv")
pseudo_1 = pd.read_csv("../../data/pseudo/pseudo_fold_1.csv")
pseudo_2 = pd.read_csv("../../data/pseudo/pseudo_fold_2.csv")
pseudo_3 = pd.read_csv("../../data/pseudo/pseudo_fold_3.csv")
pseudo_4 = pd.read_csv("../../data/pseudo/pseudo_fold_4.csv")

In [ ]:
pseudo_0["cohesion"] = (pseudo_0["cohesion"] + pseudo_1["cohesion"] + pseudo_2["cohesion"] + pseudo_3["cohesion"] + pseudo_4["cohesion"]) / 5  
pseudo_0["syntax"] = (pseudo_0["syntax"] + pseudo_1["syntax"] + pseudo_2["syntax"] + pseudo_3["syntax"] + pseudo_4["syntax"]) / 5
pseudo_0["vocabulary"] = (pseudo_0["vocabulary"] + pseudo_1["vocabulary"] + pseudo_2["vocabulary"] + pseudo_3["vocabulary"] + pseudo_4["vocabulary"]) / 5
pseudo_0["phraseology"] = (pseudo_0["phraseology"] + pseudo_1["phraseology"] + pseudo_2["phraseology"] + pseudo_3["phraseology"] + pseudo_4["phraseology"]) / 5
pseudo_0["grammar"] = (pseudo_0["grammar"] + pseudo_1["grammar"] + pseudo_2["grammar"] + pseudo_3["grammar"] + pseudo_4["grammar"]) / 5
pseudo_0["conventions"] = (pseudo_0["conventions"] + pseudo_1["conventions"] + pseudo_2["conventions"] + pseudo_3["conventions"] + pseudo_4["conventions"]) / 5

In [ ]:
pseudo_0.to_csv("../../data/pseudo/pseudo_fold_mean.csv", index=False)

In [ ]:
pd.read_csv("../../data/pseudo/pseudo_fold_mean.csv")

In [ ]:
pseudo_0["mean"] = np.mean(pseudo_0.iloc[:,2:], axis=1)
pseudo_1["mean"] = np.mean(pseudo_1.iloc[:,2:], axis=1)
pseudo_2["mean"] = np.mean(pseudo_2.iloc[:,2:], axis=1)
pseudo_3["mean"] = np.mean(pseudo_3.iloc[:,2:], axis=1)
pseudo_4["mean"] = np.mean(pseudo_4.iloc[:,2:], axis=1)

In [ ]:
summary = pd.concat([pseudo_0["mean"], pseudo_1["mean"], pseudo_2["mean"], pseudo_3["mean"], pseudo_4["mean"]], axis=1)

In [ ]:
describe = summary.T.describe()

In [ ]:
summary

In [ ]:
cnt = 0
for i in describe.iloc[2]:
    if i <= 0.01:
        print(i)
        cnt += 1
print(cnt)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
hist, bins = np.histogram(describe.iloc[2], bins=1000)

In [ ]:
X = []
for i in range(1, len(bins)):
    X.append((bins[i-1]+bins[i])/2)

In [ ]:
plt.hist(describe.iloc[2])

In [ ]:
mean_of_mean = (pseudo_0["mean"] + pseudo_1["mean"] + pseudo_2["mean"] + pseudo_3["mean"] + pseudo_4["mean"]) / 5

In [ ]:
from tqdm import tqdm
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    AddedToken
)
import pandas as pd

In [ ]:
def text_features(df, col):
    df[f"{col}_num_words"] = df[col].apply(
        lambda x: len(str(x).split())
    )  # num_words count

    df[f"{col}_num_unique_words"] = df[col].apply(
        lambda x: len(set(str(x).split()))
    )  # num_unique_words count

    df[f"{col}_num_chars"] = df[col].apply(lambda x: len(str(x)))  # num_chars count

    """
    df[f"{col}_num_stopwords"] = df[col].apply(
        lambda x: len(
            [w for w in str(x).lower().split() if w in stopwords.words("english")]
        )
    )  # stopword count
    """

    df[f"{col}_num_punctuations"] = df[col].apply(
        lambda x: len([c for c in str(x) if c in list(string.punctuation)])
    )  # num_punctuations count 句読点

    df[f"{col}_num_words_upper"] = df[col].apply(
        lambda x: len([w for w in str(x) if w.isupper()])
    )  # num_words_upper count 大文字

    df[f"{col}_mean_word_len"] = df[col].apply(
        lambda x: np.mean([len(w) for w in x.split()])
    )  # mean_word_len

    df[f"{col}_num_paragraphs"] = df[col].apply(
        lambda x: len(list(filter(None, re.split("\n\n|\r\n|\r|\n", x))))
    )  # num_paragraphs count

    """
    df[f"{col}_num_contractions"] = df[col].apply(
        contraction_count
    )  # num_contractions count

    df[f"{col}_polarity"] = df[col].apply(
        lambda x: TextBlob(x).sentiment[0]
    )  # TextBlob 感情分析

    df[f"{col}_subjectivity"] = df[col].apply(
        lambda x: TextBlob(x).sentiment[1]
    )  # TextBlob 感情分析

    df[
        [
            f"{col}_nn_count",
            f"{col}_pr_count",
            f"{col}_vb_count",
            f"{col}_jj_count",
            f"{col}_uh_count",
            f"{col}_cd_count",
        ]
    ] = df[col].apply(
        pos_count
    )  # pos count 品詞
    """

    return df

In [ ]:
def check_error(df, col):
    df["GRAMMAR"] = 0 #
    df["TYPOS"] = 0 #
    df["TYPOGRAPHY"] = 0 #
    #df["REDUNDANCY"] = 0
    df["PUNCTUATION"] = 0 #
    #df["STYLE"] = 0
    #df["MISC"] = 0
    df["CASING"] = 0 #
    #df["CONFUSED_WORDS"] = 0
    #df["COLLOCATIONS"] = 0
    #df["NONSTANDARD_PHRASES"] = 0
    #df["BRITISH_ENGLISH"] = 0
    #df["SEMANTICS"] = 0
    #df["COMPOUNDING"] = 0
    #df["AMERICAN_ENGLISH_STYLE"] = 0
    df["cnt_error"] = 0

    use_category = ["GRAMMAR", "TYPOS", "TYPOGRAPHY", "PUNCTUATION", "CASING"]

    for idx, text in enumerate(tqdm(df[col])):
        for error in tl.check(text):
            if error.category in use_category:
                df.loc[idx, error.category] += 1
            df.loc[idx, "cnt_error"] += 1

    return df


In [ ]:
import pandas as pd
import string
import numpy as np
import re
from tqdm import tqdm

In [ ]:
import language_tool_python
tl = language_tool_python.LanguageTool('en-US')

In [ ]:
train = pd.read_csv("../../data/input/train.csv")

In [ ]:
from spellchecker import SpellChecker
spell = SpellChecker()

In [ ]:
str = train["full_text"][0]

In [ ]:
def check_spell(df, col):
    spell = SpellChecker()
    df["misspelled"] = 0
    for idx, text in enumerate(tqdm(df[col])):
        text = re.split("[\n\n|\r\n|\r|\n| |!|#|$|%|&|(|)|*|+|,|-|.|/|:|;|<|=|>|?|@|\|^|_|`|{|}|~|\"|\||\[|\]]", text)
        misspelled = spell.unknown(text)
        misspelled.remove("")
        df.loc[idx, "misspelled"] = len(misspelled)

In [ ]:
a = [[]] * 5

In [ ]:
a[0]

In [ ]:
test = '"a" oh yes'

In [ ]:
re.split("[\n\n|\r\n|\r|\n| |!|#|$|%|&|(|)|*|+|,|-|.|/|:|;|<|=|>|?|@|\|^|_|`|{|}|~|']", test)

In [ ]:
str2 = re.split("[\n\n|\r\n|\r|\n| |!|#|$|%|&|(|)|*|+|,|-|.|/|:|;|<|=|>|?|@|\|^|_|`|{|}|~|\"|\||\[|\]]", str)

In [ ]:
str2

In [ ]:
misspelled = spell.unknown(str2)

In [ ]:
misspelled.remove("")

In [ ]:
misspelled

In [ ]:
len(misspelled)

In [ ]:
test = pd.read_csv("../../data/input/test.csv")

In [ ]:
a = text_features(train, "full_text")

In [ ]:
b = text_features(test, "full_text")

In [ ]:
a = check_error(train, "full_text")

In [ ]:
b = check_error(test, "full_text")

In [ ]:
b

In [ ]:
b.iloc[:, -15:].sum()

In [ ]:
a.iloc[:, -15:].sum()

In [ ]:
df

In [ ]:
for i in train["full_text"]:
    print(i)

In [ ]:
test = train["full_text"][0]

In [ ]:
tl.check(test)[1]

In [ ]:
a = {}
a["b"] = 1

In [ ]:
a

In [ ]:
cnt_check = {}
cnt_check["GRAMMAR"] = 0
cnt_check["TYPOS"] = 0
cnt_check["TYPOGRAPHY"] = 0
cnt_check["REDUNDANCY"] = 0
cnt_check["PUNCTUATION"] = 0
cnt_check["STYLE"] = 0
cnt_check["MISC"] = 0
cnt_check["CASING"] = 0
cnt_check["CONFUSED_WORDS"] = 0

In [ ]:
for i in tl.check(test):
    print(i.category)
    cnt_check[i.category] += 1

In [ ]:
cnt_check

In [ ]:
print()

In [ ]:
import textstat

In [ ]:
print(textstat.automated_readability_index(tes))

In [ ]:
print(tes)

In [ ]:
textstat.text_standard(tes)

In [ ]:
tes = train["full_text"][0]

In [ ]:
4.71 * (train["full_text_num_chars"][10] / train["full_text_num_words"][10]) + 0.5 * (train["full_text_num_words"][10] / 20) -21.43

In [ ]:
tes

In [ ]:
str(tes).split("\n")

In [ ]:
b = re.split("[\n\n|\r\n|\r|\n]", tes)

In [ ]:
b

In [ ]:
from nltk.corpus import stopwords

In [ ]:
import string

In [ ]:
for i in tes.split(". "):
    print("-----")
    print(i)

In [ ]:
string.punctuation

In [ ]:
tes

In [ ]:
re.sub("\n\n|\r\n|\r|\n", " ", tes).replace("  ", " ")

In [ ]:
re.split("\n\n|\r\n|\r|\n", tes)

In [ ]:
list(filter(None, re.split("\n\n|\r\n|\r|\n", tes)))

In [ ]:
for i in train["full_text"][:10]:
    print(i)

In [ ]:
def preprocess_text(data):
    for i, t in enumerate(data["full_text"]):
        data.loc[i, "full_text"] = data.loc[i, "full_text"].replace("\n\n", "[BR]")
        data.loc[i, "full_text"] = data.loc[i, "full_text"].replace("\r\n", "[BR]")
        #data.loc[i, "full_text"] = data.loc[i, "full_text"].replace("\n", "[BR]")
        #data.loc[i, "full_text"] = data.loc[i, "full_text"].replace("\r", "[BR]")
    return data

In [ ]:
def preprocess_text(data):
    for i, t in enumerate(data["full_text"]):
        data["full_text"][i] = data["full_text"][i].replace("\n\n", " ")
        data["full_text"][i] = data["full_text"][i].replace("\r\n", " ")
        data["full_text"][i] = data["full_text"][i].replace("\n", " ")
        data["full_text"][i] = data["full_text"][i].replace("\r", " ")
    return data

In [ ]:
preprocess_text(train)

In [ ]:
train["full_text"][0]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

In [ ]:
tokenizer.add_tokens("[BR]", special_tokens=True)
#tokenizer.lf_token_id = tokenizer.get_added_vocab()['\n']

In [ ]:
ids = tokenizer(train["full_text"][0], add_special_tokens=True)["input_ids"]

In [ ]:
tokenizer("[BR]", add_special_tokens=False)

In [ ]:
tokenizer.convert_ids_to_tokens(ids)

In [ ]:
train["full_text"][9]

In [ ]:
string.punctuation

In [ ]:
for i, t in enumerate(train["full_text"]):
    if "#" in t:
        print(i)

In [ ]:
train["full_text"][20]

In [ ]:
cnt = 0
for i, t in enumerate(train["full_text"]):
    if "\n\n" in t:
        # print(i)
        cnt += 1
print(cnt)

In [ ]:
cnt = 0
for i, t in enumerate(train["full_text"]):
    if "" in t:
        # print(i)
        cnt += 1
print(cnt)

In [ ]:
3709 - 3667

In [ ]:
train["full_text"][3852]

In [ ]:
text

In [ ]:
text.replace("\n", "")

In [ ]:
tokenizer(text, add_special_tokens=False)

In [ ]:
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import sklearn.metrics as metrics
import torch
import torch.nn as nn
import torch.optim as optim
from pytorch_lightning import Trainer, seed_everything
from tqdm import tqdm
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
)

In [ ]:
# Datasets
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, cfg, X, y=None):
        self.cfg = cfg
        if y is None:
            self.X = X.values
            self.y = torch.zeros(len(self.X), dtype=torch.float32)
        else:
            self.X = X.values
            self.y = torch.tensor(y.values, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        X = self._prepare_input(self.X[index])
        y = self.y[index]
        return X, y

    def _prepare_input(self, X):
        X = self.cfg["tokenizer"].encode_plus(
            X,
            return_tensors=None,
            add_special_tokens=True,
            max_length=self.cfg["tokenizer_params"]["max_length"],
            padding="max_length",
            truncation=True,
        )
        for k, v in X.items():
            X[k] = torch.tensor(v, dtype=torch.long)
        return X

In [ ]:
def collate(inputs):
        # 一番長いtokenへ合わせる
        mask_len = int(inputs["attention_mask"].sum(axis=1).max())
        for k, v in inputs.items():
            inputs[k] = inputs[k][:, :mask_len]
        return inputs

In [ ]:
# read csv
test = pd.read_csv("../../data/input/test.csv")
test = test["full_text"]

In [ ]:
cfg = { "test_loader":{
    "batch_size": 4,
    "shuffle": False,
    #"num_workers": 2,
    "pin_memory": True,
    "drop_last": False
    },
    "tokenizer_params":{
        "max_length": 512
    }
}

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
cfg["tokenizer"] = tokenizer
test_dataset = TableDataset(cfg, test)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    **cfg["test_loader"],
)

In [ ]:
for step, (inputs, labels) in enumerate(test_dataloader):
    print(step)
    print(inputs)
    print(labels)

In [ ]:
tr_config = AutoConfig.from_pretrained(
        "microsoft/deberta-v3-base", output_hidden_states=True
    )

In [ ]:
tr_config.hidden_size

In [ ]:
model = AutoModel.from_pretrained(
                "microsoft/deberta-v3-base", config=tr_config
            )

In [ ]:
layers = [model.embeddings] + list(model.encoder.layer)

In [ ]:
layers

In [ ]:
len(layers)

In [ ]:
import math

In [ ]:
lr_decay = 0.5**(1./len(layers))

In [ ]:
lr_decay

In [ ]:
lr_decay**len(layers)

In [ ]:
ｃ

In [ ]:
num_layers = model.config.num_hidden_layers

In [ ]:
num_layers

In [ ]:
layers.reverse()

In [ ]:
0.95**13

In [ ]:
len(layers)

In [ ]:
model.named_parameters

In [ ]:
tr_config

In [ ]:
debert = AutoModel.from_pretrained("microsoft/deberta-v3-base", config=tr_config)

In [ ]:
outputs = debert(**inputs)

In [ ]:
torch.cat([outputs["hidden_states"][-1*i][:,0] for i in range(1, 4+1)], dim=1)

In [ ]:
outputs["hidden_states"][0][:,0].size()

In [ ]:
for i in debert.named_parameters():
    print(i)

In [ ]:
cfg = {
    "general":{
        "seed": 42,
        "cv": True,
        "fold": [0, 1, 2, 3, 4], # list (0-idx start) or null. set one element list, hold-out mode.
        "n_splits": 5,
        "save_name": "submission_roberta"
    },

    "model": "transformers",
    "model_name": "microsoft/deberta-v3-base",
    "pretrained": True,

    "header": "mean_pooling",

    "tokenizer_params":{
        "max_length": 1429
    },

    "transformers_params":{
        "gradient_checkpointing": True,
        "encoder_lr": 2.0e-5,
        "decoder_lr": 2.0e-5,
        "weight_decay": 0.01,
        "num_cycles": 0.5,
        "num_warmup_steps": 0,
        "scheduler": "cosine"
    },

    "pl_params":{
        "max_epochs": 4,
        "accelerator": "auto",
        "accumulate_grad_batches": 1,
        "precision": 16, # 16 or 32
        "deterministic": False,
        "benchmark": False,
        "enable_checkpointing": False,
        "enable_model_summary": False,
        "enable_progress_bar": True,
        "logger": False
        # "limit_train_batches": 0.01, # for debug
        # "limit_val_batches": 0.05, # for debug
    },

    "early_stopping": None,

    "criterion": "SmoothL1Loss",

    "test_loader":{
        "batch_size": 4,
        "shuffle": False,
        "num_workers": 2,
        "pin_memory": True,
        "drop_last": False
    }
}

In [ ]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = (
            attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        )
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

In [ ]:
class TransformersModel(pl.LightningModule):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.criterion = nn.__dict__[cfg["criterion"]]()

        self.tr_config = AutoConfig.from_pretrained(
            cfg["model_name"], output_hidden_states=True
        )
        self.tr_config.hidden_dropout = 0.0
        self.tr_config.hidden_dropout_prob = 0.0
        self.tr_config.attention_dropout = 0.0
        self.tr_config.attention_probs_dropout_prob = 0.0
        if cfg["pretrained"]:
            self.model = AutoModel.from_pretrained(
                cfg["model_name"], config=self.tr_config
            )
        else:
            self.model = AutoModel(self.tr_config)
        if self.cfg["transformers_params"]["gradient_checkpointing"]:
            self.model.gradient_checkpointing_enable()

        # header
        if cfg["header"] == "mean_pooling":
            self.pool = (MeanPooling(),)
            self.fc = nn.Linear(self.tr_config.hidden_size, 6)
        elif cfg["header"] == "cls_cancatenate":
            self.fc = (nn.Linear(self.tr_config.hidden_size * 4, 6),)

        self._init_weights(self.fc)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.tr_config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.tr_config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def feature(self, inputs):
        outputs = self.model(**inputs)
        if self.cfg["header"] == "mean_pooling":
            last_hidden_states = outputs[0]
            feature = self.pool(last_hidden_states, inputs["attention_mask"])
        elif self.cfg["header"] == "cls_cancatenate":
            feature = torch.cat(
                [outputs["hidden_states"][-1 * i][:, 0] for i in range(1, 4 + 1)], dim=1
            )
        return feature

    def forward(self, x):
        x = self.feature(x)
        x = self.fc(x)
        return x

    def collate(self, inputs):
        # 一番長いtokenへ合わせる
        mask_len = int(inputs["attention_mask"].sum(axis=1).max())
        for k, v in inputs.items():
            inputs[k] = inputs[k][:, :mask_len]
        return inputs

    def training_step(self, batch, batch_idx):
        X, y = batch
        X = self.collate(X)
        pred_y = self.forward(X).squeeze()
        loss = self.criterion(pred_y, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def training_epoch_end(self, outputs):
        loss_list = [x["loss"] for x in outputs]
        avg_loss = torch.stack(loss_list).mean()
        self.log("train_avg_loss", avg_loss, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        X, y = batch
        X = self.collate(X)
        pred_y = self.forward(X).squeeze()
        loss = self.criterion(pred_y, y)
        self.log("valid_loss", loss, prog_bar=True)
        return {"valid_loss": loss, "preds": pred_y, "targets": y}

    def validation_epoch_end(self, outputs):
        loss_list = [x["valid_loss"] for x in outputs]
        preds = torch.cat([x["preds"] for x in outputs], dim=0).cpu().detach().numpy()
        targets = (
            torch.cat([x["targets"] for x in outputs], dim=0).cpu().detach().numpy()
        )
        avg_loss = torch.stack(loss_list).mean()
        score, scores = mcrmse(targets, preds)
        self.log("valid_avg_loss", avg_loss, prog_bar=True)
        self.log("valid_score", score, prog_bar=True)
        return avg_loss

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        X, _ = batch
        X = self.collate(X)
        pred_y = self.forward(X)
        return pred_y

    def get_scheduler(self, optimizer, num_train_steps):
        if self.cfg["transformers_params"]["scheduler"] == "linear":
            scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=self.cfg["transformers_params"]["num_warmup_steps"],
                num_training_steps=num_train_steps,
            )
        elif self.cfg["transformers_params"]["scheduler"] == "cosine":
            scheduler = get_cosine_schedule_with_warmup(
                optimizer,
                num_warmup_steps=self.cfg["transformers_params"]["num_warmup_steps"],
                num_training_steps=num_train_steps,
                num_cycles=self.cfg["transformers_params"]["num_cycles"],
            )
        return scheduler

    def configure_optimizers(self):
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {
                "params": [
                    p
                    for n, p in self.model.named_parameters()
                    if not any(nd in n for nd in no_decay)
                ],
                "lr": self.cfg["transformers_params"]["encoder_lr"],
                "weight_decay": self.cfg["transformers_params"]["weight_decay"],
            },
            {
                "params": [
                    p
                    for n, p in self.model.named_parameters()
                    if any(nd in n for nd in no_decay)
                ],
                "lr": self.cfg["transformers_params"]["encoder_lr"],
                "weight_decay": 0.0,
            },
            {
                "params": [p for n, p in self.named_parameters() if "model" not in n],
                "lr": self.cfg["transformers_params"]["decoder_lr"],
                "weight_decay": 0.0,
            },
        ]
        optimizer = optim.AdamW(
            optimizer_parameters,
            lr=self.cfg["transformers_params"]["encoder_lr"],
        )

        scheduler = self.get_scheduler(
            optimizer, self.trainer.estimated_stepping_batches
        )
        scheduler = {"scheduler": scheduler, "interval": "step", "frequency": 1}
        return [optimizer], [scheduler]

In [ ]:
model = TransformersModel(cfg)

In [ ]:
for i in model.model.named_parameters():
    print(i[0])

In [ ]:
len(outputs["hidden_states"])

In [ ]:
outputs[0].size()

In [ ]:
bert.config

In [ ]:
import pandas as pd

In [ ]:
import pathlib

In [ ]:
df = pd.read_csv("../../data/input/train.csv")

In [ ]:
df

In [ ]:
df2 = pd.read_csv("../../data/input/train.csv").drop(["cohesion", "syntax", "vocabulary", "phraseology", "grammar", "conventions"], axis=1)

In [ ]:
df2

In [ ]:
value = df.loc[:, ["cohesion", "syntax", "vocabulary", "phraseology", "grammar", "conventions"]].values

In [ ]:
df2["cohesion"] = value[:, 0]
df2["syntax"] = value[:, 1]
df2["vocabulary"] = value[:, 2]
df2["phraseology"] = value[:, 3]
df2["grammar"] = value[:, 4]
df2["conventions"] = value[:, 5]

In [ ]:
df2

In [ ]:
df

In [ ]:
input_dir = "../../data/input/fb1_train"

In [ ]:
txt_list = list(pathlib.Path(input_dir).glob('**/*.txt'))

In [ ]:
length = len(txt_list)

In [ ]:
length

In [ ]:
txt_list_1 = txt_list[:3000]
txt_list_2 = txt_list[3000:6000]
txt_list_3 = txt_list[6000:9000]
txt_list_4 = txt_list[9000:12000]
txt_list_5 = txt_list[12000:]

In [ ]:
len(txt_list_1) + len(txt_list_2) + len(txt_list_3) + len(txt_list_4) + len(txt_list_5)

In [ ]:
from tqdm import tqdm
import gc

In [ ]:
fb1_train_text_id1 = []
fb1_train_full_text1 = []
for i in tqdm(txt_list_1):
    with open(i, encoding="utf_8") as f:
        text_id = str(i).split("\\")[-1][:-4]
        fb1_train_text_id1.append(text_id)
        fb1_train_full_text1.append(f.read())

In [ ]:
gc.collect()
fb1_train_text_id2 = []
fb1_train_full_text2 = []
for i in tqdm(txt_list_2):
    with open(i, encoding="utf_8") as f:
        text_id = str(i).split("\\")[-1][:-4]
        fb1_train_text_id2.append(text_id)
        fb1_train_full_text2.append(f.read())

In [ ]:
gc.collect()
fb1_train_text_id3 = []
fb1_train_full_text3 = []
for i in tqdm(txt_list_3):
    with open(i, encoding="utf_8") as f:
        text_id = str(i).split("\\")[-1][:-4]
        fb1_train_text_id3.append(text_id)
        fb1_train_full_text3.append(f.read())

In [ ]:
gc.collect()
fb1_train_text_id4 = []
fb1_train_full_text4 = []
for i in tqdm(txt_list_4):
    with open(i, encoding="utf_8") as f:
        text_id = str(i).split("\\")[-1][:-4]
        fb1_train_text_id4.append(text_id)
        fb1_train_full_text4.append(f.read())

In [ ]:
gc.collect()
fb1_train_text_id5 = []
fb1_train_full_text5 = []
for i in tqdm(txt_list_5):
    with open(i, encoding="utf_8") as f:
        text_id = str(i).split("\\")[-1][:-4]
        fb1_train_text_id5.append(text_id)
        fb1_train_full_text5.append(f.read())

In [ ]:
len(fb1_train_text_id)

In [ ]:
len(fb1_train_full_text)

In [ ]:
fb2_train_df = pd.DataFrame(
        data={"text_id": fb2_train_text_id, "full_text": fb2_train_full_text}
    )

In [ ]:
fb2_train_df.to_csv(f"../../data/input/fb2_train_.csv", index=False)

In [ ]:
fb1 = pd.read_csv(f"../../data/input/fb1_train_.csv")

In [ ]:
fb1["fb"] = 1

In [ ]:
fb1

In [ ]:
fb2 = pd.read_csv(f"../../data/input/fb2_train_.csv")

In [ ]:
fb2["fb"] = 2

In [ ]:
fb2

In [ ]:
fb3 = pd.read_csv(f"../../data/input/train.csv").drop(["cohesion", "syntax", "vocabulary", "phraseology", "grammar", "conventions"], axis=1)

In [ ]:
fb3["fb"] = 3

In [ ]:
fb3

In [ ]:
fb = pd.concat([fb3, fb1, fb2]).reset_index(drop=True)

In [ ]:
fb

In [ ]:
fb_nodup = fb[~fb["text_id"].duplicated()].reset_index(drop=True)

In [ ]:
fb_nodup

In [ ]:
fb_nodup[fb_nodup["fb"] == 3]

In [ ]:
fb_nodup[fb_nodup["full_text"].duplicated()]